In [1]:
import sqlite3
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image
from tkinter import messagebox
import gspread
import os
import re
import json
import webbrowser
from tkinter import font
from datetime import datetime
import tkinter.simpledialog as simpledialog

from oauth2client.service_account import ServiceAccountCredentials
# Create the main window
root = tk.Tk()
root.title("Student Portal")
root.geometry("400x250")

conn = sqlite3.connect('database.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        username TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL
    )
''')
conn.close()
username = ""
def open_google_form():
    # Define the URL of the Google Form
    google_form_url ="https://docs.google.com/forms/d/e/1FAIpQLSfunN2S1MuYtdOEXJquLYcVA2BGiDVpcxi8uEZFlIT460ws3A/viewform"

    # Append the username to the URL as a query parameter
    google_form_url += "?username={}".format(username)

    # Open the Google Form in a web browser
    webbrowser.open(google_form_url)

def open_link(link):
    webbrowser.open(link)

def open_projects_window():
    # Load credentials from JSON file
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('portal.json', scope)
    client = gspread.authorize(creds)

    # Open the Google Form responses spreadsheet
    sheet = client.open('PROJECT GOOGLE FORMS').sheet1

    # Get all values from the spreadsheet
    values = sheet.get_all_values()

    # Store the original data for filtering
    original_data = values[1:]  # Exclude header row

    def filter_projects():
        query = search_entry.get()
        branch = branch_var.get()
        year = year_var.get()

        # Clear the treeview
        projects_treeview.delete(*projects_treeview.get_children())

        # Filter projects based on the query, branch, and year
        filtered_projects = [row for row in original_data if
                             (query.lower() in ' '.join(row).lower()) and
                             (branch == "" or row[5] == branch) and
                             (year == "" or row[6] == year)]

        # Display filtered projects in the treeview
        for i, row in enumerate(filtered_projects, start=1):
            projects_treeview.insert('', tk.END, values=row, tags=('Button',))

    # Create a new window for displaying the projects
    projects_window = tk.Toplevel(root)
    projects_window.title('Projects')
    projects_window.geometry('800x600')

    # Search Bar
    search_frame = tk.Frame(projects_window)
    search_frame.pack(padx=10, pady=10)

    # Count Labels
    count_frame = tk.Frame(projects_window)
    count_frame.pack(padx=10, pady=(0, 10))

    total_projects_label = tk.Label(count_frame, text="Total Projects: 0")
    total_projects_label.pack(side=tk.LEFT, padx=10)

    search_label = tk.Label(search_frame, text="Search:")
    search_label.pack(side=tk.LEFT)

    search_entry = tk.Entry(search_frame, width=30)
    search_entry.pack(side=tk.LEFT, padx=5)

    def search_projects():
        filter_projects()

    search_button = tk.Button(search_frame, text="Search", command=search_projects)
    search_button.pack(side=tk.LEFT)

    # Dropdown Menu (Combobox)
    branch_label = tk.Label(search_frame, text="Branch:")
    branch_label.pack(side=tk.LEFT)

    branches = set(row[4] for row in original_data)
    branch_var = tk.StringVar()
    branch_dropdown = ttk.Combobox(search_frame, textvariable=branch_var, values=[""] + list(branches))
    branch_dropdown.pack(side=tk.LEFT)
    branch_dropdown.bind('<<ComboboxSelected>>', lambda event: filter_projects())

    # Dropdown Menu (Combobox)
    year_label = tk.Label(search_frame, text="Year:")
    year_label.pack(side=tk.LEFT)

    years = set(row[5] for row in original_data)
    year_var = tk.StringVar()
    year_dropdown = ttk.Combobox(search_frame, textvariable=year_var, values=[""] + list(years))
    year_dropdown.pack(side=tk.LEFT)
    year_dropdown.bind('<<ComboboxSelected>>', lambda event: filter_projects())

    # Scrollbars
    scrollbar_y = tk.Scrollbar(projects_window, orient='vertical')
    scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

    scrollbar_x = tk.Scrollbar(projects_window, orient='horizontal')
    scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

    # Update total projects count label
    total_projects = len(original_data)
    total_projects_label.config(text=f"Total Projects: {total_projects}")

    # Create a treeview to display data
    projects_treeview = ttk.Treeview(
        projects_window, yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set, height=20)

    projects_treeview.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
    scrollbar_y.config(command=projects_treeview.yview)
    scrollbar_x.config(command=projects_treeview.xview)

    headers = values[0]
    projects_treeview['columns'] = headers

    for i, header in enumerate(headers):
        projects_treeview.heading(i, text=header, anchor=tk.CENTER)
        projects_treeview.column(i, width=150, anchor=tk.CENTER)

    for i, row in enumerate(original_data, start=1):
        projects_treeview.insert('', tk.END, values=row, tags=('Button',))

    # Add a tag configuration to bind the button click event
    projects_treeview.tag_configure('Button', background='white')
    projects_treeview.tag_bind('Button', '<1>', lambda event, row=row: open_link(row[-1]))
    count_label_students = tk.Label(search_frame, text="Total Students: 0")
    count_label_students.pack(side=tk.LEFT)
    total_students = len(set([row[1] for row in values[1:]]))  # Count unique student names in column 1
    count_label_students.config(text=f"Total Students: {total_students}")

    def update_total_projects_count():
        total_projects_label.config(text=f"Total Projects: {len(original_data)}")

    def add_project(row):
        projects_treeview.insert('', tk.END, values=row, tags=('Button',))
        original_data.append(row)
        update_total_projects_count()

    def delete_project(row):
        index = original_data.index(row)
        projects_treeview.delete(index)
        original_data.remove(row)
        update_total_projects_count()

    def search_projects(query):
        filter_projects()

    # Focus the search entry for user convenience
    search_entry.focus_set()

    # Branch Count Labels
    branch_count_labels = {}
    year_count_labels = {}

    # Count projects for each branch
    for row in original_data:
        branch_count_labels[row[6]] = branch_count_labels.get(row[6], 0) + 1

    # Update branch count labels
    for branch, count in branch_count_labels.items():
        branch_count_label = tk.Label(count_frame, text=f"{branch}: {count}")
        branch_count_label.pack(side=tk.LEFT, padx=10)

    # Count projects for each year
    for row in original_data:
        year_count_labels[row[7]] = year_count_labels.get(row[7], 0) + 1

    # Update year count labels
    for year, count in year_count_labels.items():
        year_count_label = tk.Label(count_frame, text=f"{year}: {count}")
        year_count_label.pack(side=tk.LEFT, padx=10)

event_window = None
event_data = None


def retrieve_data(login_email):
    conn = sqlite3.connect('login_credentials.db')
    cursor = conn.cursor()

    query = "SELECT * FROM users WHERE email = ?"
    cursor.execute(query, (login_email,))

    row = cursor.fetchone()

    cursor.close()
    conn.close()

    if row is not None:
        return row
    else:
        return None

def show_profile(login_email):
    profile_window = tk.Toplevel()
    profile_window.title('Profile Details')

    profile_data = [None]  # Encapsulate profile_data inside a list

    def display_profile_data(profile_data):
        if profile_data[0] is not None:
            email_label.config(text="Email: " + (profile_data[0][1] or ""))
            username_label.config(text="Username: " + (profile_data[0][2] or ""))
            branch_label.config(text="Branch: " + (profile_data[0][4] or ""))
            section_label.config(text="Section: " + (profile_data[0][5] or ""))
            phone_label.config(text="Phone: " + (profile_data[0][9] or ""))


            linkedin_url = profile_data[0][7]
            linkedin_url_label.config(text="LinkedIn: " + (linkedin_url or ""), fg="blue", cursor="hand2")

            github_url = profile_data[0][8]
            github_url_label.config(text="GitHub: " + (github_url or ""), fg="blue", cursor="hand2")
        else:
            messagebox.showerror("Error", "Profile data not found")

    email_label = tk.Label(profile_window, text="Email: ")
    email_label.grid(row=0, column=0, sticky="w")

    username_label = tk.Label(profile_window, text="Username: ")
    username_label.grid(row=1, column=0, sticky="w")

    branch_label = tk.Label(profile_window, text="Branch: ")
    branch_label.grid(row=2, column=0, sticky="w")

    section_label = tk.Label(profile_window, text="Section: ")
    section_label.grid(row=3, column=0, sticky="w")

    phone_label = tk.Label(profile_window, text="Phone: ")
    phone_label.grid(row=4, column=0, sticky="w")

    linkedin_url_label = tk.Label(profile_window, text="LinkedIn: ", fg="blue", cursor="hand2")
    linkedin_url_label.grid(row=5, column=0, sticky="w")

    github_url_label = tk.Label(profile_window, text="GitHub: ", fg="blue", cursor="hand2")
    github_url_label.grid(row=6, column=0, sticky="w")

    def open_linkedin_url():
        linkedin_url = profile_data[0][7]
        if linkedin_url:
            webbrowser.open_new(linkedin_url)

    def open_github_url():
        github_url = profile_data[0][8]
        if github_url:
            webbrowser.open_new(github_url)

    def profile_button_click():
        profile_data[0] = retrieve_data(login_email)
        display_profile_data(profile_data)

    # Load and display the initial profile data
    profile_data[0] = retrieve_data(login_email)
    display_profile_data(profile_data)

    # Bind the functions to the corresponding labels
    linkedin_url_label.bind("<Button-1>", lambda e: open_linkedin_url())
    github_url_label.bind("<Button-1>", lambda e: open_github_url())

    root.withdraw()


def user_dashboard(login_email):
    global background_photo
    dashboard_window = tk.Toplevel(root)
    dashboard_window.title("Dashboard")
    dashboard_window.geometry("800x600")

    def user_events():
        event_window = tk.Toplevel(dashboard_window)
        event_window.title("Event Details")
        event_window.geometry("900x700")

        def load_event_details_user():
            # Load the event details from the JSON file
            if os.path.exists("event_details.json"):
                with open("", "r") as f:
                    event_data = json.load(f)

                # Clear the text widget
                event_text.delete("1.0", tk.END)

                # Get the description and hyperlinks from the event data dictionary
                description = event_data['description']
                hyperlinks = event_data['hyperlinks']

                # Insert the description into the text widget
                event_text.insert(tk.END, description)

                # Configure the text widget to recognize and style hyperlinks
                for hyperlink in hyperlinks:
                    start_index = '1.0'
                    while True:
                        start_index = event_text.search(hyperlink, start_index, stopindex=tk.END, regexp=True)
                        if not start_index:
                            break
                        end_index = f"{start_index}+{len(hyperlink)}c"
                        event_text.tag_add("hyperlink", start_index, end_index)
                        start_index = end_index

        def on_link_click_user(event):
            # Get the clicked text and its index
            index = event.widget.index("@%s,%s" % (event.x, event.y))
            tags = event.widget.tag_names(index)
            if "hyperlink" in tags:
                # Get the text of the clicked hyperlink
                hyperlink_text = event.widget.get("current linestart", "current lineend")

                # Extract the URL from the clicked hyperlink using regular expressions
                url_match = re.search(r'\bhttps?://\S+', hyperlink_text)
                if url_match:
                    url = url_match.group(0)

                    # Open the URL in the default web browser
                    webbrowser.open(url)

        event_label = tk.Label(event_window, text="Event Details:")
        event_label.pack()

        event_text = tk.Text(event_window, height=40, width=100)  # Adjust the height and width as needed
        event_text.pack()

        # Configure the text widget to recognize and style hyperlinks
        event_text.tag_configure("hyperlink", foreground="blue", underline=True)
        event_text.tag_bind("hyperlink", "<Button-1>", on_link_click_user)

        # Load and display the initial event details in the user dashboard
        load_event_details_user()

        def check_event_file():
            global event_data  # Use the event_data defined in the outer function
            if os.path.exists("event_details.json"):
                with open("event_details.json", "r") as f:
                    event_data_new = json.load(f)
                    if event_data_new != event_data:
                        event_data.clear()
                        event_data.update(event_data_new)
                        load_event_details_user()
            dashboard_window.after(1000, check_event_file)

        # Start checking the event file for changes
        check_event_file()

        def go_back():
            event_window.withdraw()
            dashboard_window.deiconify()

        # Add back button
        back_button = tk.Button(event_window, text="Back", command=go_back)
        back_button.place(x=620, y=500)

    menubar = tk.Menu(dashboard_window)
    dashboard_window.config(menu=menubar)
    menubar.add_command(label="        ")  # Add spaces as a label

    # Create the Events menu
    events_menu = tk.Menu(menubar, tearoff=0)
    events_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Events", menu=events_menu)
    events_menu.add_command(label="Show Events", command=user_events)
    menubar.add_command(label="        ")  # Add spaces as a label

    # Create the Projects menu
    projects_menu = tk.Menu(menubar, tearoff=0)
    projects_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Projects", menu=projects_menu)
    projects_menu.add_command(label="View Projects", command=open_projects_window)
    projects_menu.add_command(label="Upload Projects", command=open_google_form)
    menubar.add_command(label="        ")  # Add spaces as a label

    # Create the Logout menu
    logout_menu = tk.Menu(menubar, tearoff=0)
    logout_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Logout", menu=logout_menu)
    logout_menu.add_command(label="Logout", command=lambda: (root.deiconify(), dashboard_window.withdraw()))

    # Create the Profile menu
    profile_menu = tk.Menu(menubar, tearoff=0)
    profile_menu.configure(font=font.Font(size=16))
    menubar.add_cascade(label="Profile", menu=profile_menu)
    profile_menu.add_command(label="My Profile", command=lambda: show_profile(login_email))

    dashboard_window.mainloop()


# Call the user_dashboard function to start the dashboard




def open_registration_window():
    registration_window = tk.Toplevel()
    registration_window.title("Registration")
    registration_window.geometry("400x350")

    username_label = tk.Label(registration_window, text="Username:")
    username_label.pack()

    username_entry = tk.Entry(registration_window)
    username_entry.pack()

    email_label = tk.Label(registration_window, text="Email:")
    email_label.pack()

    email_entry = tk.Entry(registration_window)
    email_entry.pack()

    password_label = tk.Label(registration_window, text="Password:")
    password_label.pack()

    password_entry = tk.Entry(registration_window, show="*")
    password_entry.pack()

    branch_label = tk.Label(registration_window, text="Branch:")
    branch_label.pack()

    branch_entry = tk.Entry(registration_window)
    branch_entry.pack()

    section_label = tk.Label(registration_window, text="Section:")  # Add section label
    section_label.pack()

    section_entry = tk.Entry(registration_window)  # Add section entry
    section_entry.pack()
    
    phone_label = tk.Label(registration_window, text="Phone Number:")
    phone_label.pack()

    phone_entry = tk.Entry(registration_window)
    phone_entry.pack()


    
    linkedin_label = tk.Label(registration_window, text="LinkedIn URL:")
    linkedin_label.pack()

    linkedin_entry = tk.Entry(registration_window)
    linkedin_entry.pack()

    github_label = tk.Label(registration_window, text="GitHub URL:")
    github_label.pack()

    github_entry = tk.Entry(registration_window)
    github_entry.pack()

    

    def register():
        global back_photo, register_photo,username

        # Function to handle registration
        name = username_entry.get()
        email = email_entry.get()
        password = password_entry.get()
        branch = branch_entry.get()
        section = section_entry.get()
        phone_number = phone_entry.get()
        linkedin_url = linkedin_entry.get()
        github_url = github_entry.get()
      

    # Insert values into the database
    
        if name == "" or email == "" or password == "" or branch == "" or section == ""or phone_number == ""or linkedin_url=="" or github_url==""   :
            messagebox.showerror("Error", "Please fill all the fields")
            return

        conn = sqlite3.connect('login_credentials.db')
        cursor = conn.cursor()
        cursor.execute("INSERT INTO users (name, email, password, branch, section, linkedin_url, github_url, phone_number)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                   (name, email, password, branch, section, linkedin_url, github_url, phone_number))
        
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "User registered successfully")

        username_entry.delete(0, tk.END)
        email_entry.delete(0, tk.END)
        password_entry.delete(0, tk.END)
        branch_entry.delete(0, tk.END)
        section_entry.delete(0, tk.END)  # Clear section entry
          # Clear roll number entry

    register_button = tk.Button(registration_window, text="Register", command=register)
    register_button.pack()


        

    # Create a back button to go back to the login window
    # Create a back button to go back to the login window
    back_image = Image.open("back-button.png")
    back_image = back_image.resize((60, 50))  # Resize the image
    back_photo = ImageTk.PhotoImage(back_image)
    back_button = tk.Button(frame, image=back_photo, command=lambda:(root.deiconify(),registration_window.destroy()),borderwidth =0, highlightthickness=0, relief=tk.FLAT)
    back_button.image = back_photo
    back_button.grid(row=4, column=0, padx=10, pady=20)
    # Load the register image
    register_image = Image.open("register_but.png")
    register_image = register_image.resize((80, 50))  # Resize the image
    register_photo = ImageTk.PhotoImage(register_image)

    # Create a register button
    register_button = tk.Button(frame, image=register_photo, command=open_registration_window,
                                borderwidth=0, highlightthickness=0)
    register_button.image = register_photo
    register_button.grid(row=4, column=1, padx=60, pady=40,sticky="e")
def open_admin_dashboard():
    global event_entry, username

    # Create the dashboard window
    dashboard = tk.Toplevel(root)
    dashboard.title("Dashboard")
    dashboard.geometry("800x600")

    event_window = None

    def show_events():
        nonlocal event_window

        # Check if the event file exists
        if os.path.exists("event_details.txt"):
            # Read the event details from the file
            with open("event_details.txt", "r") as f:
                event_details = f.read()

            if event_window is not None and event_window.winfo_exists():
                # If the event window already exists, bring it to the front
                event_window.lift()
            else:
                # Create a new event window to display the event details
                event_window = tk.Toplevel(dashboard)
                event_window.title("Event Details")
                event_window.geometry("900x700")

                def go_back():
                    event_window.withdraw()
                    dashboard.deiconify()

                def save_event_details():
                    # Get the tagged text from the text widget
                    tagged_text = event_text.get("1.0", tk.END)

                    # Extract the hyperlinks from the tagged text
                    hyperlinks = re.findall(r'\bhttps?://\S+', tagged_text, flags=re.IGNORECASE)

                    # Create an event dictionary with the tagged text and hyperlinks
                    event = {
                        'description': tagged_text,
                        'hyperlinks': hyperlinks
                    }

                    # Save the event details as JSON
                    with open("event_details.json", "w") as f:
                        json.dump(event, f)
                        root.withdraw()
                    # Display a success message
                    messagebox.showinfo("Success", "Event details saved successfully!")

                def load_event_details():
                    # Load the event details from the JSON file
                    if os.path.exists("event_details.json"):
                        with open("event_details.json", "r") as f:
                            event = json.load(f)

                        # Clear the text widget
                        event_text.delete("1.0", tk.END)

                        # Get the description and hyperlinks from the event dictionary
                        description = event['description']
                        hyperlinks = event['hyperlinks']

                        # Insert the description into the text widget
                        event_text.insert(tk.END, description)

                        # Configure the text widget to recognize and style hyperlinks
                        for hyperlink in hyperlinks:
                            start_index = '1.0'
                            while True:
                                start_index = event_text.search(hyperlink, start_index, stopindex=tk.END, regexp=True)
                                if not start_index:
                                    break
                                end_index = f"{start_index}+{len(hyperlink)}c"
                                event_text.tag_add("hyperlink", start_index, end_index)
                                start_index = end_index
                def on_link_click(event):
                    # Get the clicked text and its index
                    index = event.widget.index("@%s,%s" % (event.x, event.y))
                    tags = event.widget.tag_names(index)
                    if "hyperlink" in tags:
                        # Get the text of the clicked hyperlink
                        hyperlink_text = event.widget.get("current linestart", "current lineend")

                        # Extract the URL from the clicked hyperlink using regular expressions
                        url_match = re.search(r'\bhttps?://\S+', hyperlink_text)
                        if url_match:
                            url = url_match.group(0)

                            # Open the URL in the default web browser
                            webbrowser.open(url)

                def add_hyperlink():
                    # Prompt the admin to enter the URL
                    url = simpledialog.askstring("Add Hyperlink", "Enter the URL:")

                    if url:
                        # Insert the URL as a hyperlink
                        event_text.insert(tk.END, url, "hyperlink")
                        event_text.insert(tk.END, " ")

                # Display the event details
                event_label = tk.Label(event_window, text="Event Details:")
                event_label.pack()

                event_text = tk.Text(event_window, height=40, width=100)  # Adjust the height and width as needed
                event_text.pack()

                # Insert event details into the text widget
                event_text.insert(tk.END, event_details)

                # Configure the text widget to recognize and style hyperlinks
                event_text.tag_configure("hyperlink", foreground="blue", underline=True)
                event_text.tag_bind("hyperlink", "<Button-1>", on_link_click)
                # Load and display the event details
                load_event_details()

                # Add the current date label
                current_date = datetime.now().strftime("%B %d")
                date_label = tk.Label(event_window, text="Date: " + current_date)
                date_label.pack()

                # Add hyperlink button
                add_hyperlink_button = tk.Button(event_window, text="Add Hyperlink", command=add_hyperlink)
                add_hyperlink_button.pack()

                # Add save button
                save_button = tk.Button(event_window, text="Save", command=save_event_details)
                save_button.place(x=800, y=600)

                # Add back button
                back_button = tk.Button(event_window, text="Back", command=go_back)
                back_button.place(x=920, y=600)

        else:
            messagebox.showinfo("No Event Details", "No event details found.")

    def logout():
        dashboard.destroy()
        root.deiconify()
    # Create the menu bar
    menubar = tk.Menu(dashboard)
    dashboard.config(menu=menubar)
    # Add empty labels to increase the menubar size
    menubar.add_command(label="        ")  # Add spaces as a label

    # Create the Events menu
    events_menu = tk.Menu(menubar, tearoff=0)
    # Adjust the font size as desired
    events_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Events", menu=events_menu)
    events_menu.add_command(label="Show Events", command=show_events)
    menubar.add_command(label="        ")  # Add dashes as a label






    # Create the Projects menu
    projects_menu = tk.Menu(menubar, tearoff=0)
    projects_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Projects", menu=projects_menu)
    projects_menu.add_command(label="View Projects", command=open_projects_window)
    menubar.add_command(label="        ")  # Add dashes as a label


    # Create the File menu
    # Create the Logout menu
    logout_menu = tk.Menu(menubar, tearoff=0)
    logout_menu.configure(font=font.Font(size=16))  # Adjust the font size as desired

    menubar.add_cascade(label="Logout", menu=logout_menu)
    logout_menu.add_command(label="Logout", command=logout)





    # Add an image to the dashboard
    image = Image.open("logo.jpg")
    image = image.resize((1400, 700), Image.LANCZOS)
    photo = ImageTk.PhotoImage(image)
    image_label = tk.Label(dashboard, image=photo)
    image_label.image = photo
    image_label.pack()
    dashboard.mainloop()

def login():
    global username_entry, password_entry  # Declare the variables as global

    # Retrieve the entered username and password
    entered_username = username_entry.get()
    entered_password = password_entry.get()

    # Check if the username and password are empty
    if not entered_username or not entered_password:
        messagebox.showerror("Error", "Please enter both username and password.")
        return

    # Retrieve the user details from the database using the entered username
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username=?", (entered_username,))
    user = cursor.fetchone()
    conn.close()

    # Check if a user with the entered username exists and the password is correct
    if user and entered_password == user[4]:
        username = entered_username
        user_dashboard()
    else:
        messagebox.showerror("Error", "Invalid username or password.")
# Create the username and password Entry widgets

def collect_credentials():
    email = email_entry.get()
    password = password_entry.get()
    
    # Check if the credentials match with admin accounts
    if (email == "sandeep.g@bvrithyderabad.edu.in" and password == "myportal12") or (email == "p2@gmail.com" and password == "p2"):
        open_admin_dashboard()
    else:
        conn = sqlite3.connect('login_credentials.db')
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users WHERE email = ? AND password = ?", (email, password))
        result = cursor.fetchone()
        conn.close()

    if result is None:
        messagebox.showerror("Error", "Invalid email or password")
    else:
        email_entry.delete(0, tk.END)
        password_entry.delete(0, tk.END)
            # Open the dashboard window

        user_dashboard(email)


background_image = Image.open(r"C:\Users\Akula Akila\Downloads\logo.jpg")
background_image = background_image.resize((900, 800), Image.LANCZOS)
background_photo = ImageTk.PhotoImage(background_image)

# Create a full-screen Canvas widget and place the background image on it
canvas = tk.Canvas(root, width=root.winfo_screenwidth(), height=root.winfo_screenheight())
canvas.pack()
canvas.create_image(0, 0, image=background_photo, anchor=tk.NW)

# Add header label
head_label = tk.Label(root, text="Welcome to Student Portal", bg="light green", font=("Helvetica", 20, "bold"))
head_label.place(relx=0.75, rely=0.15, anchor=tk.CENTER)

frame = tk.Frame(root, bg="light blue")
frame.place(relx=0.75, rely=0.5, anchor=tk.CENTER)

# Add the email and password labels and entries to the frame
email_label = tk.Label(frame, text="Email:", font=("Helvetica", 16), bg="light blue")
email_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

email_entry = tk.Entry(frame, font=("Helvetica", 16), bg="light blue")
email_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")

password_label = tk.Label(frame, text="Password:", font=("Helvetica", 16), bg="light blue")
password_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

password_entry = tk.Entry(frame, show="*", font=("Helvetica", 16), bg="light blue")
password_entry.grid(row=1, column=1, padx=5, pady=5, sticky="e")

# Add register labels and buttons
register_label = tk.Label(frame, text="Don't have an account?", font=("Helvetica", 12), bg="light blue")
register_label.grid(row=4, column=0, padx=10, pady=5, sticky="se")

register_button = tk.Button(frame, text="SignUp", font=("Helvetica", 16), bg="light green",
                            command=open_registration_window)
register_button.grid(row=3, column=0, padx=15,pady=20)


submit_button = tk.Button(frame, text="Login", font=("Helvetica", 16), bg="light green", command=collect_credentials)
submit_button.grid(row=3, column=1, columnspan=2, pady=20)

# Start the main event loop
root.mainloop()

In [1]:
{
  "VoilaConfiguration": {
    "tornado_settings": {
      "static_handler_args": {
        "whitelist": ["/voila/files/favicon.ico"]
      }
    }
  }
}


{'VoilaConfiguration': {'tornado_settings': {'static_handler_args': {'whitelist': ['/voila/files/favicon.ico']}}}}

In [2]:
{
  "VoilaConfiguration": {
    "show_tracebacks": true
  }
}


NameError: name 'true' is not defined